# Explore SEC API & download Income Statement and Balance Sheets

In [1]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None # not to get SettingWithCopyWarning

## Set variables - Ticker, CIK etc.

In [2]:
cik = '0000051253'
ticker = 'IFF'
e_mail = 'zs.nagy.1989@gmail.com'

## Function to use

In [3]:
def update_us_gaap_jsons(path, cik, dictionary_new_key):
    import json
    # open existing JSON file
    with open(path) as infile:
        data = json.load(infile)
    # add new key to the dictionary
    if cik in data.keys():
        print('CIK is already in JSON file.')
    else:
        data[cik] = dictionary_new_key[cik]
        print('New CIK has been added to JSON file.')
    # write & update old JSON file
    with open(path, 'w') as outfile:
        json.dump(data, outfile, sort_keys=False, indent=2)

In [4]:
def get_company_reports(cik=str, report_list=['10-K', '10-Q'], e_mail=str):
    # create link
    cik = cik
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    # create header
    header = {
    "User-Agent": e_mail#, # remaining fields are optional
    #    "Accept-Encoding": "gzip, deflate",
    #    "Host": "data.sec.gov"
    }
    # get company fillings
    company_filings = requests.get(url, headers=header).json()
    # create dataframe & filter based on documentum type
    company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
    company_filings_df_filt = company_filings_df.loc[company_filings_df['form'].isin(report_list)]
    # reorder the documents
    company_filings_df_filt = company_filings_df_filt.reindex(index=company_filings_df_filt.index[::-1])
    # drop original index
    company_filings_df_filt.reset_index(inplace=True, drop=True)
    company_filings_df_filt = company_filings_df_filt[['reportDate', 'form']].copy().rename(columns={'reportDate':'end'})
    # add cik column
    company_filings_df_filt['cik'] = [cik for x in range(len(company_filings_df_filt))]

    return company_filings_df_filt

In [5]:
def insert_fourth_quaterly_report(input_df=pd.DataFrame):
    counter = 0
    data = {
        'end': [],
        'form': [],
        'cik': []   
        }

    for index, value in enumerate(input_df.form):
        if (counter == 3 and value == '10-K') or (index < 3 and value == '10-K'):
            data['form'].append('10-Q')
            data['form'].append(value)
            data['end'].append(input_df.end.iloc[index])
            data['end'].append(input_df.end.iloc[index])
            data['cik'].append(input_df.cik.iloc[index])
            data['cik'].append(input_df.cik.iloc[index])
        else:
            data['form'].append(value)
            data['end'].append(input_df.end.iloc[index])
            data['cik'].append(input_df.cik.iloc[index])
        # calculate 10-Q rows
        if value == '10-Q':
            counter += 1
        else:
            counter = 0

    result = pd.DataFrame(data)
    return result

In [6]:
def get_income_statements(cik=str, e_mail=str, input_df=pd.DataFrame, us_gaap_dict=dict):
    # create link
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    
    # create header
    header = {
    "User-Agent": e_mail
    }
    
    # get company fillings
    company_facts = requests.get(url, headers=header).json()
    # create output template from company fillings table
    result = input_df.copy()
    result['end'] = pd.to_datetime(result['end'])
    # iterate us_gaap keys related to specific cik (company)
    for key in us_gaap_dict.keys():
        # set key unit parameters based on filling type
        if key == 'shares':
            word = 'shares'
        elif key == 'eps_dil':
            word = 'USD/shares'
        else:
            word = 'USD'

        # empty DataFrame that will be filled and after merged with result
        data = pd.DataFrame()
        # iterate us-gaap lists
        for index in range(len(us_gaap_dict[key])):
            # get data & create dataframe
            temporary = pd.DataFrame(company_facts["facts"]["us-gaap"][us_gaap_dict[key][index]]["units"][word])
            # set date columns datatype
            temporary['end'] = pd.to_datetime(temporary['end'])
            temporary['start'] = pd.to_datetime(temporary['start'])
            # calculate the covered timeperiods of unique reports
            temporary['time_diff'] = temporary['end'] - temporary['start']
            temporary['time_diff'] = [x.days for x in temporary['time_diff']]
            # filter based on report lenght
            temporary = temporary.loc[((temporary['time_diff'] > 345) & (temporary['time_diff'] < 390)) | ((temporary['time_diff'] > 80) & (temporary['time_diff'] < 100))]
            # set form based on report lenght
            temporary['form'] = ['10-K' if elem > 120 else '10-Q' for elem in temporary['time_diff']]
            temporary.sort_values(by=['end'])
            # drop duplicates
            data.drop_duplicates(subset=['val', 'form'], keep='last', inplace=True, ignore_index=True)
            data.drop_duplicates(subset=['end', 'form'], keep='last', inplace=True, ignore_index=True)
            # merge temporary data before add to result
            if index == 0:
                data = temporary.loc[temporary['form'].isin(['10-Q', '10-K'])]
            else:
                data = pd.concat([data, temporary], ignore_index=True)
        
        # add data to result
        data['end'] = pd.to_datetime(data['end'])
        result = pd.merge_asof(result.sort_values('end'), data[['val', 'end', 'form']].sort_values('end'), on="end" , by="form")

        result.rename(columns={'val': key}, inplace=True)
    
    if 'inpairm' in result.columns:
        result['inpairm'].fillna(0.0, inplace=True)

    return result

In [7]:
def fill_missing_annual_depr(input_df=pd.DataFrame):
    # fill missing annual D&A
    for i, val in enumerate(input_df['depr_amort']):
        if input_df['form'].iloc[i] == '10-K' and i > 0:
            input_df['depr_amort'].iloc[i] = 4 * input_df['depr_amort'].iloc[i-1]
        elif input_df['form'].iloc[i] == '10-K' and i == 0:
            input_df['depr_amort'].iloc[i] = 4 * input_df['depr_amort'].iloc[i+1]

    return input_df

In [8]:
def get_balance_sheet(cik=str, e_mail=str, input_df=pd.DataFrame, us_gaap_dict=dict):
    # create link
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    
    # create header
    header = {
    "User-Agent": e_mail
    }
    
    # get company fillings
    company_facts = requests.get(url, headers=header).json()
    # create output template from company fillings table
    result = input_df.copy()
    result['end'] = pd.to_datetime(result['end'])
    # iterate us_gaap keys related to specific cik (company)
    for key in us_gaap_dict.keys():
        # empty DataFrame that will be filled and after merged with result
        data = pd.DataFrame()
        # iterate us-gaap lists
        for index in range(len(us_gaap_dict[key])):
            # get data & create dataframe
            temporary = pd.DataFrame(company_facts["facts"]["us-gaap"][us_gaap_dict[key][index]]["units"]["USD"])
            # set date columns datatype
            # drop duplicates
            data.drop_duplicates(subset=['val', 'form'], keep='last', inplace=True, ignore_index=True)
            data.drop_duplicates(subset=['end', 'val'], keep='last', inplace=True, ignore_index=True)
            # merge temporary data before add to result
            if index == 0:
                data = temporary.loc[temporary['form'].isin(['10-Q', '10-K'])]
            else:
                data = pd.concat([data, temporary], ignore_index=True)
        
        # add data to result
        data['end'] = pd.to_datetime(data['end'])
        result = pd.merge_asof(result.sort_values('end'), data[['val', 'end']].sort_values('end'), on="end")

        result.rename(columns={'val': key}, inplace=True)

    return result

## 0. US-GAAP dictionaries

In [9]:
us_gaap_is = {
    cik:{
        'rev': ['RevenueFromContractWithCustomerExcludingAssessedTax'],
        'cost_sale': ['CostOfGoodsAndServicesSold'],
        'sale_gen_adm': ['SellingGeneralAndAdministrativeExpense'],
        'inpairm': ['ImpairmentOfLongLivedAssetsHeldForUse'],
        'op_income': ['OperatingIncomeLoss'],
        'int_income': ['OtherNonoperatingIncomeExpense'],
        'int_exp': ['InterestExpense'],
        'inc_tax_exp': ['IncomeTaxExpenseBenefit'],
        'net_inc': ['ProfitLoss'],
        'shares': ['WeightedAverageNumberOfDilutedSharesOutstanding'],              # not ['USD'] but ['shares']
        'eps_dil': ['EarningsPerShareDiluted']                                      # not ['USD'] but ['USD/shares']
        }
    }

In [10]:
us_gaap_depr = {
    cik:{
        'depr_amort': ['DepreciationDepletionAndAmortization']
        }
    }

In [11]:
us_gaap_bs = {
    cik:{
        'cash_cash_eq': ['CashAndCashEquivalentsAtCarryingValue'],
        'acc_rec': ['ReceivablesNetCurrent'],
        'invent': ['InventoryNet'],
        'other_curr_ass': ['OtherAssetsCurrent'],
        'tot_curr_ass': ['AssetsCurrent'],
        'ppe_net': ['PropertyPlantAndEquipmentNet'],
        'goodwill': ['Goodwill'],
        'intang_ass': ['FiniteLivedIntangibleAssetsNet'],
        'other_ass': ['OtherAssetsNoncurrent'],
        'tot_ass': ['Assets'],
        'short_debt': ['DebtCurrent'],
        'acc_pay': ['AccountsPayableCurrent'],
        'other_curr_liab': ['OtherLiabilitiesCurrent'],
        'tot_curr_liab': ['LiabilitiesCurrent'],
        'tot_long_debt': ['LongTermDebtNoncurrent'],
        'other_liab': ['OtherLiabilitiesNoncurrent'],
        'tot_equity': ['StockholdersEquity']
        }
    }

### Useful LINK
#### https://www.kaggle.com/code/svendaj/extracting-data-from-sec-edgar-restful-apis

## 1. Get Comnpany related report list

In [12]:
input_df = get_company_reports(
    cik=cik,
    report_list=['10-K', '10-Q'],
    e_mail=e_mail
    )
input_df.head()

,end,form,cik
0,2018-06-30,10-Q,0000051253
1,2018-09-30,10-Q,0000051253
2,2018-12-31,10-K,0000051253
3,2019-03-31,10-Q,0000051253
4,2019-06-30,10-Q,0000051253


In [13]:
input_df_inserted = insert_fourth_quaterly_report(input_df)
input_df_inserted.head()

,end,form,cik
0,2018-06-30,10-Q,0000051253
1,2018-09-30,10-Q,0000051253
2,2018-12-31,10-Q,0000051253
3,2018-12-31,10-K,0000051253
4,2019-03-31,10-Q,0000051253


## 2. Filling Income Sattement

In [14]:
df_income_statement = get_income_statements(
    cik=cik,
    e_mail=e_mail,
    input_df=input_df_inserted,
    us_gaap_dict=us_gaap_is[cik]
    )

df_income_statement.head()

,end,form,cik,rev,cost_sale,sale_gen_adm,inpairm,op_income,int_income,int_exp,inc_tax_exp,net_inc,shares,eps_dil
0,2018-06-30,10-Q,0000051253,920016000,521299000,157407000,0.0,154509000,20655000,53246000,22769000,99685000,79303000,1.25
1,2018-09-30,10-Q,0000051253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,4986000,95716000,81647000,1.17
2,2018-12-31,10-Q,0000051253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,4986000,95716000,81647000,1.17
3,2018-12-31,10-K,0000051253,3977539000,2294832000,707461000,0.0,583882000,35243000,132558000,107976000,339781000,88121000,3.79
4,2019-03-31,10-Q,0000051253,1297402000,766143000,213182000,0.0,163870000,7278000,36572000,23362000,111214000,113389000,0.96


## 2.b Filling Deplation, Amortzation, Depratiation

In [15]:
df_income_amortization = get_income_statements(
    cik=cik,
    e_mail=e_mail,
    input_df=input_df_inserted,
    us_gaap_dict=us_gaap_depr[cik]
    )

# fill missing annual D&A
df_income_amortization = fill_missing_annual_depr(df_income_amortization)

df_income_amortization.head()

,end,form,cik,depr_amort
0,2018-06-30,10-Q,0000051253,33384000
1,2018-09-30,10-Q,0000051253,33384000
2,2018-12-31,10-Q,0000051253,33384000
3,2018-12-31,10-K,0000051253,133536000
4,2019-03-31,10-Q,0000051253,81775000


## 3. Filling Balance Sheet

In [16]:
df_balance_sheet = get_balance_sheet(
    cik=cik,
    e_mail=e_mail,
    input_df=input_df_inserted,
    us_gaap_dict=us_gaap_bs[cik]
    )

df_balance_sheet.head()

,end,form,cik,cash_cash_eq,acc_rec,invent,other_curr_ass,tot_curr_ass,ppe_net,goodwill,intang_ass,other_ass,tot_ass,short_debt,acc_pay,other_curr_liab,tot_curr_liab,tot_long_debt,other_liab,tot_equity
0,2018-06-30,10-Q,0000051253,322423000,723855000,695192000,285110000,2026580000,867629000,1148586000,391426000,157017000,4673442000,6500000,315656000,263448000,699370000,1717189000,238635000,1749855000
1,2018-09-30,10-Q,0000051253,5274459000,701111000,719508000,251749000,6946827000,874817000,1152864000,385575000,167978000,9615542000,45985000,312236000,257364000,758456000,4331242000,235332000,4031814000
2,2018-12-31,10-Q,0000051253,634897000,63460000,1078537000,277036000,2941860000,1241152000,5378388000,3039322000,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000
3,2018-12-31,10-K,0000051253,634897000,63460000,1078537000,277036000,2941860000,1241152000,5378388000,3039322000,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000
4,2019-03-31,10-Q,0000051253,483504000,1003965000,1114488000,310243000,2925825000,1294029000,5434000000,2974177000,583389000,13211420000,84003000,476413000,414626000,1144134000,4421430000,492029000,6143211000


## 4. Merge & Check every statement

In [17]:
merged_temp = pd.merge(df_income_statement, df_income_amortization, on=['end', 'form', 'cik'])
merged_final = pd.merge(merged_temp, df_balance_sheet, on=['end', 'form', 'cik'])
merged_final['ticker'] = [ticker for x in range(len(merged_final.end))]
merged_final.head()

,end,form,cik,rev,cost_sale,sale_gen_adm,inpairm,op_income,int_income,int_exp,...,other_ass,tot_ass,short_debt,acc_pay,other_curr_liab,tot_curr_liab,tot_long_debt,other_liab,tot_equity,ticker
0,2018-06-30,10-Q,0000051253,920016000,521299000,157407000,0.0,154509000,20655000,53246000,...,157017000,4673442000,6500000,315656000,263448000,699370000,1717189000,238635000,1749855000,IFF
1,2018-09-30,10-Q,0000051253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,...,167978000,9615542000,45985000,312236000,257364000,758456000,4331242000,235332000,4031814000,IFF
2,2018-12-31,10-Q,0000051253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,...,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000,IFF
3,2018-12-31,10-K,0000051253,3977539000,2294832000,707461000,0.0,583882000,35243000,132558000,...,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000,IFF
4,2019-03-31,10-Q,0000051253,1297402000,766143000,213182000,0.0,163870000,7278000,36572000,...,583389000,13211420000,84003000,476413000,414626000,1144134000,4421430000,492029000,6143211000,IFF


In [18]:
# write file
merged_final.to_csv('../sec_report_csv/{}_{}_sec_reports.csv'.format(cik, ticker), index=False)

## 5. Update JSON files with US-GAAP codes

You should manually cross check the downloaded data, and after that change the variable

In [28]:
downloaded_data_correct = False

In [29]:
if downloaded_data_correct:
    # unpdate Income Statement - US-GAAP 
    update_us_gaap_jsons(
        path='../JSON/us_gaap_is.json',
        cik=cik,
        dictionary_new_key=us_gaap_is
    )
    # unpdate D&A - US-GAAP 
    update_us_gaap_jsons(
        path='../JSON/us_gaap_depr.json',
        cik=cik,
        dictionary_new_key=us_gaap_depr
    )
    # unpdate Balance Sheet - US-GAAP
    update_us_gaap_jsons(
        path='../JSON/us_gaap_bs.json',
        cik=cik,
        dictionary_new_key=us_gaap_bs
    )